In [1]:
import sys

sys.path.append("/Users/PRVATE/Documents/tf_transformers/src/")

# This is a dummy Model .

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

from transformers import TFT5Model
from tf_transformers.models import T5Encoder
from tf_transformers.models import EncoderDecoder

from tf_transformers.core import LegacyModule

from tf_transformers.utils import convert_t5_hf_to_tf_transformers
from tf_transformers.data import pad_dataset

from tf_transformers.text import TextDecoderSeq2Seq
from tf_transformers.text import TextDecoderSerializableSeq2Seq

import json
import time
import os

# Load tf_transformers Model

Configs are in the `model_configs` folder in the root of the repo.


We will be using `convert_albert_hf_to_tf_transformers` function.

Always use `is_training=False` to load the model and pass this model for conversion.

Do not enablle `pipeline_mode='auto-regressive` while converting. Because, variable name

differs due to `tf.cond` usage. 

# Steps:

1. Load a model using **`is_training=False`**

2. Convert it using conversion functions from `tf_transformers.utils`

3. Save the `checkpoint` .

4. For auto-regressive tasks (text generation) use **`pipeline_mode='auto-regressive`**

   along with **`is_training=False`** and load from the checkpoint.

# T5 Specifics
1. T5 is an **Seq2Seq ( Encoder Decoder ) Model** . It has an encoder part and decoder part.

2. In tf_transformers, you can convert any **Encoder to Decoder** with few keyword arguments (`is_decoder=True,`).

In [3]:
# Load tf_transformers model
# Most config we will be providing

# Default configs for the model

model_config_dir = '/Users/PRVATE/Documents/tf_transformers/model_configs/'
model_name = 't5_small'
config_location = os.path.join(model_config_dir, model_name, 'config.json')
config = json.load(open(config_location))

# Always do this



# Encoder

config["bidirectional"] = True
config["mask_mode"] = "user_defined"
encoder_layer = T5Encoder(
    config=config, mask_mode=config["mask_mode"], is_training=False, name="t5_encoder"
)


# Decoder

# T5 needs bidirectional = False
# Decoder mask mode has to be causal (auto-regressive)
# encoder_embedding_layer (share embedding from encoder)
config["bidirectional"] = False
config["mask_mode"] = "causal"
decoder_layer = T5Encoder(
    config=config,
    name="t5_decoder",
    mask_mode=config["mask_mode"],
    is_decoder=True,
    is_training=False,
    share_encoder_embeddings=True,
    encoder_embedding_layer=encoder_layer._embedding_layer,
)

INFO:absl:We are overwriding `is_training` is False to `is_training` to                     True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:We are overwriding `is_training` is False to `is_training` to                     True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_hidden_states ---> Tensor("encoder_hidden_states:0", shape=(None, None, 512), dtype=float32)
INFO:absl:decoder_encoder_mask ---> Tensor("decoder_encoder_mask:0", shape=(None, None, None), dtype=float32)
INFO:absl:Initialized Variables


# Create T5 (Encoder Decoder) model
We will use above initialized (randomly) encoder and decoder layer and pass it to `EncoderDecoder` API

**Note: If you want to use EncoderDecoder, for text-generation tasks, make sure `decoder_layer` must
have `is_training=False` and `pipeline_mode='auto-regressive'` enabled. 
After that, set `is_training=False` in `EncoderDecoder` model**

In [4]:
# Train mode (Keras Layer/Legacy Layer)

t5_enc_dec_model = EncoderDecoder(
    encoder=encoder_layer,
    decoder=decoder_layer,
    is_training=True,
    name="t5_small",
    use_dropout=False,
)

t5_enc_dec_model = t5_enc_dec_model.get_model()

# Once this is loaded, we will delete decoder_layer , as well as t5_enc_dec_model
# Because, we want to use Albert as the decoder layer

INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)


In [5]:
# Delte
del t5_enc_dec_model
del decoder_layer

# Load GPT2 Model



In [11]:
from tf_transformers.models import GPT2Encoder
# Load tf_transformers model
# Most config we will be providing

# Default configs for the model

model_config_dir = '/Users/PRVATE/Documents/tf_transformers/model_configs/'
model_name = 'gpt2_base'
config_location = os.path.join(model_config_dir, model_name, 'config.json')
config = json.load(open(config_location))

# Always do this


# tf_transformers Layer (an extension of Keras Layer)
# This is not Keras model, but extension of keras Layer

# Save as saved_model
# If you want to use the model for Auto Regressive tasks ( text-generation ),
# you have to enable pipeline_mode='auto-regressive'.
# Because TF needs extra cache inputs in the saved_model format for doing efficient caching

# Decoder requires causal masking
config["mask_mode"] = 'causal'
model_layer = GPT2Encoder(
    config=config,
    name="gpt2",
    mask_mode=config["mask_mode"],
    is_training=False,
    is_decoder=True
)


INFO:absl:We are overwriding `is_training` is False to `is_training`                     to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_hidden_states ---> Tensor("encoder_hidden_states:0", shape=(None, None, 768), dtype=float32)
INFO:absl:decoder_encoder_mask ---> Tensor("decoder_encoder_mask:0", shape=(None, None, None), dtype=float32)
INFO:absl:Initialized Variables


# Create Encoder Decoder model

Here we use **GPT2** as decoder and **T5** as encoder 

In [12]:
# Train mode
enc_dec_model = EncoderDecoder(
    encoder=encoder_layer,
    decoder=model_layer,
    is_training=True,
    name="t5_gpt2",
    use_dropout=False,
)
enc_dec_model = enc_dec_model.get_model()
enc_dec_model.save_weights("dummy.h5")

INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)


# Create Encoder Decoder model (Auto Regressive)


In [13]:
config["mask_mode"] = 'causal'
model_layer = GPT2Encoder(
    config=config,
    name="gpt2",
    mask_mode=config["mask_mode"],
    is_training=False,
    is_decoder=True,
    pipeline_mode = 'auto-regressive'
)


INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("decoder_input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_hidden_states ---> Tensor("encoder_hidden_states_1:0", shape=(None, None, 768), dtype=float32)
INFO:absl:decoder_encoder_mask ---> Tensor("decoder_encoder_mask_1:0", shape=(None, None, None), dtype=float32)
INFO:absl:all_cache_key ---> Tensor("all_cache_key:0", shape=(None, None, 12, None, 64), dtype=float32)
INFO:absl:all_cache_value ---> Tensor("all_cache_value:0", shape=(None, None, 12, None, 64), dtype=float32)
INFO:absl:Initialized Variables


In [14]:
# Train mode
enc_dec_model_test = EncoderDecoder(
    encoder=encoder_layer,
    decoder=model_layer,
    is_training=False,
    name="t5_gpt2",
    use_dropout=False,
)
enc_dec_model_test = enc_dec_model_test.get_model()
enc_dec_model_test.load_weights("dummy.h5")

INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_all_cache_key ---> Tensor("all_cache_key:0", shape=(None, None, 12, None, 64), dtype=float32)
INFO:absl:decoder_all_cache_value ---> Tensor("all_cache_value:0", shape=(None, None, 12, None, 64), dtype=float32)
INFO:absl:encoder_hidden_states ---> Tensor("encoder_hidden_states_1:0", shape=(None, None, 768), dtype=float32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_all_cache_key -

# Tokenizer 
LegcayAI has in-built tokenizer. You can either use it. (Not recommended).

The main difference is how we handle `SPECIAL TOKENS`. Apart from that its the same.

Recommended use **HuggingFace tokenizer**

For tf_transformers tokenizer usage check **`tf_transformers/tests/notebooks/tokenizers`**

In [15]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")


# Convert text to tokens (for T5 Model)
@pad_dataset
def tokenizer_fn(tokenizer, text_list):
    """Tokenizer fn should return a dict (no padding is required).
    Make sure, you pass all primary keys required to the model

    text_list: a list of text

    {'input_ids': tf.constant([[1, 2]]),
     'input_mask': tf.constant([[1, 1]]),
     'input_type_ids': tf.constant([[1, 0]])}


    """
    input_ids = []
    input_mask = []
    for text in text_list:
        input_ids.append(tokenizer.encode(text))
        input_mask.append(tf.ones_like(input_ids[-1]).numpy().tolist())
    inputs = {"encoder_input_ids": input_ids}
    inputs["encoder_input_mask"] = input_mask
    return inputs


In [16]:
text_list = ['summarize : Hello How are you ?']

In [44]:
encoder_inputs = tokenizer_fn(tokenizer, text_list)
encoder_inputs['decoder_input_ids'] = tf.constant([[9]])
for i in range(10):
    result = enc_dec_model(encoder_inputs)
    best_id = tf.argmax(result['last_token_logits'][0])
    best_prob = tf.reduce_max(result['last_token_logits'][0])
    print(best_id, '-->', best_prob)
    encoder_inputs['decoder_input_ids'] = tf.concat([encoder_inputs['decoder_input_ids'], 
              tf.expand_dims(tf.expand_dims(tf.cast(best_id, tf.int32), 0), 1)], axis=1)

tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183208, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183208, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)


In [46]:
enc_dec_model_test.input

{'encoder_input_ids': <tf.Tensor 'input_ids:0' shape=(None, None) dtype=int32>,
 'encoder_input_mask': <tf.Tensor 'input_mask:0' shape=(None, None) dtype=int32>,
 'decoder_input_ids': <tf.Tensor 'decoder_input_ids_1:0' shape=(None, None) dtype=int32>,
 'decoder_all_cache_key': <tf.Tensor 'all_cache_key:0' shape=(None, None, 12, None, 64) dtype=float32>,
 'decoder_all_cache_value': <tf.Tensor 'all_cache_value:0' shape=(None, None, 12, None, 64) dtype=float32>,
 'encoder_hidden_states': <tf.Tensor 'encoder_hidden_states_1:0' shape=(None, None, 768) dtype=float32>}

In [59]:
encoder_inputs = tokenizer_fn(tokenizer, text_list)
encoder_inputs['decoder_input_ids'] = tf.constant([[9]])
encoder_inputs['decoder_all_cache_key'] = tf.zeros((12, 1, 12, 1, 64 ), dtype=tf.float32)
encoder_inputs['decoder_all_cache_value'] = tf.zeros((12, 1, 12, 1, 64 ), dtype=tf.float32)
encoder_inputs['encoder_hidden_states'] = tf.zeros((1, 10, 768), dtype=tf.float32)

for i in range(10):
    result = enc_dec_model_test(encoder_inputs)
    best_id = tf.argmax(result['last_token_logits'][0])
    best_prob = tf.reduce_max(result['last_token_logits'][0])
    print(best_id, '-->', best_prob)
    encoder_inputs['decoder_input_ids'] = tf.expand_dims(tf.expand_dims(tf.cast(best_id, tf.int32), 0), 1)
    encoder_inputs['decoder_all_cache_key'] = result['decoder_all_cache_key']
    encoder_inputs['decoder_all_cache_value'] = result['decoder_all_cache_value']
    encoder_inputs['encoder_hidden_states'] = result['encoder_hidden_states']


tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183208, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183208, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.0183206, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)
tf.Tensor(8310, shape=(), dtype=int64) --> tf.Tensor(2.018321, shape=(), dtype=float32)


In [60]:
!rm -rf dummy.h5